# Imports

In [1]:
import os
import re
from typing import List, Dict
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import zipfile
from pathlib import Path
import pandas as pd

# Acesso a API
- Identificação e download dos arquivos ZIP selecionados 
- Extração dos ZIPs 
- Identificação dos arquivos com Despesas com Eventos/Sinistros

### Identificação e download dos arquivos ZIP selecionados 

In [2]:
def get_years_from_home_page(URL: str, session: requests.Session) -> List[str]:
    """
    Obtém a lista de anos disponíveis na página base.
    """
    resp = session.get(URL, timeout=30)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")

    padrao_ano = re.compile(r"^(20\d{2})/$")

    anos = []
    for link in soup.find_all("a"):
        href = link.get("href", "")
        match = padrao_ano.match(href)
        if match:
            anos.append(match.group(1))

    if not anos:
        raise RuntimeError("Nenhum ano encontrado no diretório base.")

    return anos

def get_trimesters_from_page(URL: str, ano: str, session: requests.Session) -> List[Dict[str, any]]:
    """
    Obtém os 3 ultimos trimestres disponível para o ano fornecido.
    """
    url_ano = urljoin(URL, f"{ano}/")
    print(f"\nProcessando ano {ano}...")
    resp_ano = session.get(url_ano, timeout=30)
    if resp_ano.status_code != 200:
        print(f"Falha ao acessar {url_ano}")
        return []

    soup_ano = BeautifulSoup(resp_ano.text, "html.parser")
    arquivos = []

    # Para cada ZIP
    for link in soup_ano.find_all("a"):
        href = link.get("href", "")
            
        if not href.lower().endswith(".zip"):
                continue
            
        trimestre = extrair_trimestres(href)

        arquivos.append({
            "ano": ano,
            "trimestre": trimestre,
            "nome": href,
            "url": urljoin(url_ano, href)
        })

    if not arquivos:
        print(f"Nenhum ZIP encontrado para {ano}")
        return []

    # Selecionar os 3 ultimos semestres
    arquivos.sort(key=lambda x: x["trimestre"], reverse=True)
    return arquivos[:3]

def extrair_trimestres(nome_arquivo: str) -> str:
    """
    Extrai os trimestre dos nomes de arquivos ZIP.
    """
    nome = nome_arquivo.lower()

    # Encontrar trimestre no nome
    padroes_trimestre = [
        r'([1-4])\s*t',              # 1T, 1t
        r'([1-4])\s*trim',           # 1trim
        r'([1-4])\s*trimestre',      # 1trimestre
        r'([1-4])\s*[-_ ]\s*trim',   # 1-trim
    ]

    trimestre = None
    for padrao in padroes_trimestre:
        match = re.search(padrao, nome)
        if match:
            trimestre = int(match.group(1))
            break

    if trimestre is None:
        raise ValueError(f"Trimestre não encontrado no nome do arquivo: {nome_arquivo}")

    return trimestre

def download_trimesters(pasta_destino: str, session: requests.Session, ano: str, trimestres: List[Dict[str, any]]) -> None:
    pasta_ano = os.path.join(pasta_destino, ano)
    os.makedirs(pasta_ano, exist_ok=True)

    for arq in trimestres:
        destino = os.path.join(pasta_ano, arq["nome"])

        if os.path.exists(destino):
            print(f"Já existe: {arq['nome']}")
            continue

        print(f"Baixando: {arq['nome']}")

        with session.get(arq["url"], stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(destino, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)

def baixar_ultimos_3_trimestres(pasta_destino: str):
    URL = "https://dadosabertos.ans.gov.br/FTP/PDA/demonstracoes_contabeis/"
    os.makedirs(pasta_destino, exist_ok=True)

    session = requests.Session()
    session.headers.update({"User-Agent": "ANS-Crawler/1.0"})

    # 1- Acessar lista de anos
    anos = get_years_from_home_page(URL, session)

    # 2- Para cada ano obter trimestres
    for ano in anos:
        trimestres = get_trimesters_from_page(URL, ano, session)

        # 3- Baixar os trimestres selecionados
        download_trimesters(pasta_destino, session, ano, trimestres)

In [4]:
# baixar_ultimos_3_trimestres(pasta_destino= "downloads")


Processando ano 2007...
Baixando: 2007_4_trimestre.zip
Baixando: 2007_3_trimestre.zip
Baixando: 2007_2_trimestre.zip

Processando ano 2008...
Baixando: 2008_4_trimestre.zip
Baixando: 2008_3_trimestre.zip
Baixando: 2008_2_trimestre.zip

Processando ano 2009...
Baixando: 2009_4_trimestre.zip
Baixando: 2009_3_trimestre.zip
Baixando: 2009_2_trimestre.zip

Processando ano 2010...
Baixando: 2010_4_trimestre.zip
Baixando: 2010_3_trimestre.zip
Baixando: 2010_2_trimestre.zip

Processando ano 2011...
Baixando: 20120614_2011_4_trimestre.zip
Baixando: 20120614_2011_3_trimestre.zip
Baixando: 20120614_2011_2_trimestre.zip

Processando ano 2012...
Baixando: 20130416_4T2012.zip
Baixando: 20130416_3T2012.zip
Baixando: 20130416_2T2012.zip

Processando ano 2013...
Baixando: 2013-4t.zip
Baixando: 2013-3t.zip
Baixando: 2013-2t.zip

Processando ano 2014...
Baixando: 4T2014.zip
Baixando: 3T2014.zip
Baixando: 2T2014.zip

Processando ano 2015...
Baixando: 4T2015.zip
Baixando: 3T2015.zip
Baixando: 2T2015.zip



### Extração dos ZIPs

In [20]:
def extrair_zip(caminho_zip: str, pasta_destino: str):
    """ 
    Extrai um aquivo ZIP para uma pasta destino
    """
    pasta_destino = Path(pasta_destino)
    pasta_destino.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(caminho_zip, "r") as zip_ref:
        zip_ref.extractall(pasta_destino)

def extrair_todos_os_zips_da_pasta(pasta_origem: str, pasta_destino: str):
    """ 
    Extrai todos os ZIPs de uma pasta e coloca os arq extraidos para a pasta detino
    """
    pasta_origem = Path(pasta_origem)
    pasta_destino = Path(pasta_destino)

    print(f"Extraindo pasta {pasta_origem}")

    pasta_destino.mkdir(parents=True, exist_ok=True)

    for zip_path in pasta_origem.rglob("*.zip"):
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(pasta_destino)

In [22]:
# for i in range(2007, 2026):
#     extrair_todos_os_zips_da_pasta(f"downloads/{i}/", f"downloads/{i}/extraido")
# 
# extrair_todos_os_zips_da_pasta(f"downloads/2025/", f"downloads/2025/extraido")

### Identificação dos arquivos com "Despesas com Eventos/Sinistros" 
- Incrementalmente
- FILTRO: Textos que começam com ‘despesas com eventos / sinistros’, independentemente da quantidade de espaços entre as palavras, e podendo ter qualquer texto depois.
- Para os anos de 2010, 2011, 2012, 2013, 2014, 2015, 2016 não foram encontradas ocorrencias de "Despesas com Eventos/Sinistros"

In [25]:
def csv_contem(csv_path: str, coluna: str, texto_procurado: str, chunksize: int =100_000) -> bool:
    """
    Verifica se o csv contem o texto na coluna especificada
    """

    encontrou = False

    for chunk in pd.read_csv(
    csv_path,
    sep=';',
    dtype=str,
    chunksize=chunksize,
    encoding='latin1'
    ):
        if chunk[coluna].str.contains(texto_procurado, case=False, na=False).any():
            encontrou = True
            break
    
    return encontrou

In [26]:
dict = {}
for i in range(2007, 2026):
        pasta_origem = Path(f"downloads/{i}/extraido/")
        dict[i] = []
        for csv_path in pasta_origem.glob("*.csv"):
                tem = csv_contem(csv_path, 'DESCRICAO', r'^despesas\s+com\s+eventos\s*/\s*sinistros.*')
                dict[i].append(tem)

print(dict)

{2007: [True, True, True], 2008: [True, True, True], 2009: [True, True, True], 2010: [False, False, False], 2011: [False, False, False], 2012: [False, False, False], 2013: [False, False, False], 2014: [False, False, False], 2015: [False, False, False], 2016: [False, False, False], 2017: [True, True, True], 2018: [True, True, True], 2019: [True, True, True], 2020: [True, True, True], 2021: [True, True, True], 2022: [True, True, True], 2023: [True, True, True], 2024: [True, True, True], 2025: [True, True, True]}
